In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')
sub = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.nunique().to_frame()

In [ ]:
first = train[train['breath_id']==1]
second = train[train['breath_id']==2]

In [ ]:
first.plot(x='time_step', y='u_in', kind='line', figsize=(12, 3))
second.plot(x='time_step', y='u_in', kind='line', figsize=(12, 3))
first.plot(x='time_step', y='u_out', kind='line', figsize=(12, 3))
second.plot(x='time_step', y='u_out', kind='line', figsize=(12, 3))
first.plot(x='time_step', y='pressure', kind='line', figsize=(12, 3))
second.plot(x='time_step', y='pressure', kind='line', figsize=(12, 3))

In [ ]:
for df in [train, test]:
    g = df.groupby('breath_id')['u_in']
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['u_in_first']  = g.transform('first')
    df['u_in_min']    = g.transform('min')
    df['u_in_mean']   = g.transform('mean')
    df['u_in_median'] = g.transform('median')
    df['u_in_max']    = g.transform('max')
    df['u_in_last']   = g.transform('last')
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()

    df['u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['u_in_lag6'] = df['u_in'].shift(6).fillna(0)

    df['ewm_u_in_mean'] = g.ewm(halflife=10).mean()\
                           .reset_index(level=0, drop=True)
    df['ewm_u_in_std'] = g.ewm(halflife=10).std()\
                          .reset_index(level=0, drop=True)
    df['ewm_u_in_corr'] = g.ewm(halflife=10).corr()\
                           .reset_index(level=0, drop=True)
    df['rolling_10_mean'] = g.rolling(window=10, min_periods=1).mean()\
                             .reset_index(level=0, drop=True)
    df['rolling_10_max'] = g.rolling(window=10, min_periods=1).max()\
                            .reset_index(level=0, drop=True)
    df['rolling_10_std'] = g.rolling(window=10, min_periods=1).std()\
                            .reset_index(level=0, drop=True)
    df = df.fillna(0)

In [ ]:
train

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble     import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
X_train = train.drop(['pressure', 'id', 'breath_id'], axis=1)
X_test = test.drop(['id', 'breath_id'], axis=1)
y_train = train['pressure']

In [ ]:
X_train

In [ ]:
regressor  =  HistGradientBoostingRegressor(max_iter=100,
     loss="least_absolute_deviation",early_stopping=False)
regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_train)

In [ ]:
print("MAE =", mean_absolute_error(y_pred, y_train))

In [ ]:
test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')
sub = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
test['u_in_cumsum'] = (test['u_in']).groupby(test['breath_id']).cumsum()
for df in [test]:
    df['u_in_first']  = df.groupby('breath_id')['u_in'].transform('first')
    df['u_in_min']    = df.groupby('breath_id')['u_in'].transform('min')
    df['u_in_mean']   = df.groupby('breath_id')['u_in'].transform('mean')
    df['u_in_median'] = df.groupby('breath_id')['u_in'].transform('median')
    df['u_in_max']    = df.groupby('breath_id')['u_in'].transform('max')
    df['u_in_last']   = df.groupby('breath_id')['u_in'].transform('last')
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()

    g = df.groupby('breath_id')['u_in']
    df['ewm_u_in_mean'] = g.ewm(halflife=10).mean()\
                           .reset_index(level=0, drop=True)
    df['ewm_u_in_std'] = g.ewm(halflife=10).std()\
                          .reset_index(level=0, drop=True)
    df['ewm_u_in_corr'] = g.ewm(halflife=10).corr()\
                           .reset_index(level=0, drop=True)

    df['rolling_10_mean'] = g.rolling(window=10, min_periods=1).mean()\
                             .reset_index(level=0, drop=True)
    df['rolling_10_max'] = g.rolling(window=10, min_periods=1).max()\
                            .reset_index(level=0, drop=True)
    df['rolling_10_std'] = g.rolling(window=10, min_periods=1).std()\
                            .reset_index(level=0, drop=True)
    df = df.fillna(0)

In [ ]:
X_test

In [ ]:
sub["pressure"] = regressor.predict(X_test)

In [ ]:
sub.to_csv('submission.csv',index=False)
print("lala")

In [ ]:
sub

In [ ]:
mean

In [ ]:
sub["pressure"] = mean

In [ ]:
import numpy as np
import pandas as pd
import math
import time
import pickle
import argparse
import sklearn.preprocessing
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import KFold

debug = False

def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
set_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
train = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')
sub = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
def create_features(df):
    df = df.copy()
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()

    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()

    df['u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['u_in_lag4'] = df['u_in'].shift(4).fillna(0)

    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df = pd.get_dummies(df)

    g = df.groupby('breath_id')['u_in']
    df['ewm_u_in_mean'] = g.ewm(halflife=10).mean()\
                           .reset_index(level=0, drop=True)
    df['ewm_u_in_std'] = g.ewm(halflife=10).std()\
                          .reset_index(level=0, drop=True)
    df['ewm_u_in_corr'] = g.ewm(halflife=10).corr()\
                           .reset_index(level=0, drop=True)

    df['rolling_10_mean'] = g.rolling(window=10, min_periods=1).mean()\
                             .reset_index(level=0, drop=True)
    df['rolling_10_max'] = g.rolling(window=10, min_periods=1).max()\
                            .reset_index(level=0, drop=True)
    df['rolling_10_std'] = g.rolling(window=10, min_periods=1).std()\
                            .reset_index(level=0, drop=True)

    df = df.fillna(0)

    df.drop(['id', 'breath_id'], axis=1, inplace=True)
    if 'pressure' in df.columns:
        df.drop('pressure', axis=1, inplace=True)

    return df


class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y, w):
        if y is None:
            y = np.zeros(len(X), dtype=np.float32)

        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
        self.w = w.astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i], self.w[i]

In [ ]:
features = create_features(train)
rs = sklearn.preprocessing.RobustScaler()
features = rs.fit_transform(features)  # => np.ndarray

X_all = features.reshape(-1, 80, features.shape[-1])
y_all = train.pressure.values.reshape(-1, 80)
w_all = 1 - train.u_out.values.reshape(-1, 80)  # weights for the score, but not used in this notebook

input_size = X_all.shape[2]

print(len(X_all))

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size):
        hidden = [400, 300, 200, 100]
        super().__init__()
        self.lstm1 = nn.LSTM(input_size, hidden[0],
                             batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(2 * hidden[0], hidden[1],
                             batch_first=True, bidirectional=True)
        self.lstm3 = nn.LSTM(2 * hidden[1], hidden[2],
                             batch_first=True, bidirectional=True)
        self.lstm4 = nn.LSTM(2 * hidden[2], hidden[3],
                             batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(2 * hidden[3], 50)
        self.selu = nn.SELU()
        self.fc2 = nn.Linear(50, 1)
        self._reinitialize()

    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
            elif 'fc' in name:
                if 'weight' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'bias' in name:
                    p.data.fill_(0)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x, _ = self.lstm4(x)
        x = self.fc1(x)
        x = self.selu(x)
        x = self.fc2(x)

        return x

In [ ]:
model = Model(input_size)
for name, p in model.named_parameters():
    print('%-32s %s' % (name, tuple(p.shape)))

In [ ]:
criterion = torch.nn.L1Loss()

def evaluate(model, loader_val):
    tb = time.time()
    was_training = model.training
    model.eval()

    loss_sum = 0
    score_sum = 0
    n_sum = 0
    y_pred_all = []

    for ibatch, (x, y, w) in enumerate(loader_val):
        n = y.size(0)
        x = x.to(device)
        y = y.to(device)
        w = w.to(device)

        with torch.no_grad():
            y_pred = model(x).squeeze()

        loss = criterion(y_pred, y)

        n_sum += n
        loss_sum += n*loss.item()
        
        y_pred_all.append(y_pred.cpu().detach().numpy())

    loss_val = loss_sum / n_sum

    model.train(was_training)

    d = {'loss': loss_val,
         'time': time.time() - tb,
         'y_pred': np.concatenate(y_pred_all, axis=0)}

    return d

In [ ]:
nfold = 2
kfold = KFold(n_splits=nfold, shuffle=True, random_state=228)
epochs = 15
lr = 1e-3
batch_size = 1024
max_grad_norm = 1000
log = {}

for ifold, (idx_train, idx_val) in enumerate(kfold.split(X_all)):
    print('Fold %d' % ifold)
    tb = time.time()
    model = Model(input_size)
    model.to(device)
    model.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=10)

    X_train = X_all[idx_train]
    y_train = y_all[idx_train]
    w_train = w_all[idx_train]
    X_val = X_all[idx_val]
    y_val = y_all[idx_val]
    w_val = w_all[idx_val]

    dataset_train = Dataset(X_train, y_train, w_train)
    dataset_val = Dataset(X_val, y_val, w_val)
    loader_train = torch.utils.data.DataLoader(dataset_train, shuffle=True,
                         batch_size=batch_size, drop_last=True)
    loader_val = torch.utils.data.DataLoader(dataset_val, shuffle=False,
                         batch_size=batch_size, drop_last=False)

    losses_train = []
    losses_val = []
    lrs = []
    time_val = 0
    best_score = np.inf
   
    print('epoch loss_train loss_val lr time')
    for iepoch in range(epochs):
        loss_train = 0
        n_sum = 0
        
        for ibatch, (x, y, w) in enumerate(loader_train):
            n = y.size(0)
            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()

            y_pred = model(x).squeeze()

            loss = criterion(y_pred, y)
            loss_train += n*loss.item()
            n_sum += n

            loss.backward()
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

            optimizer.step()

        val = evaluate(model, loader_val)
        loss_val = val['loss']
        time_val += val['time']

        losses_train.append(loss_train / n_sum)
        losses_val.append(val['loss'])
        lrs.append(optimizer.param_groups[0]['lr'])

        print('%3d %9.6f %9.6f %7.3e %7.1f %6.1f' %
              (iepoch + 1,
               losses_train[-1], losses_val[-1], 
               lrs[-1], time.time() - tb, time_val))

        scheduler.step(losses_val[-1])


    ofilename = 'model%d.pth' % ifold
    torch.save(model.state_dict(), ofilename)
    print(ofilename, 'written')

    log['fold%d' % ifold] = {
        'loss_train': np.array(losses_train),
        'loss_val': np.array(losses_val),
        'learning_rate': np.array(lrs),
        'y_pred': val['y_pred'],
        'idx': idx_val
    }
    
    if ifold >= 1: # due to time limit
        break

In [ ]:
print("s")